In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

#PART II : Facebook's Prophet (Greykite)

##Import

Start by installing greykite library in Colab

In [2]:
# # $CHA_BEGIN
# !pip install greykite
# !pip install prophet
# # $CHA_END

In [3]:
import datetime
import pandas as pd
import plotly.express as px
from greykite.framework.templates.autogen.forecast_config import ForecastConfig, MetadataParam, ModelComponentsParam
from greykite.framework.benchmark.data_loader_ts import DataLoaderTS
from greykite.framework.templates.forecaster import Forecaster

/Users/llm/.pyenv/versions/3.10.6/envs/PDPA-virt_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Import the data

In [4]:
df_preproc = pd.read_csv("../../data/pollution/5_Clusters/cluster1_Ouest.csv")
df_raw = pd.read_csv("../../data/pollution/5_Clusters/cluster1_Ouest.csv")

In [5]:
df_preproc

,Date_time,PM25,PM10,NO2,O3,Pollution_peak,ATMO,sin_Month,cos_Month,sin_day,cos_day,confinement
0,2018-01-01,12.90,8.74,14.33,78.50,0,0,0.000000,1.000000,-0.781831,0.623490,0
1,2018-01-02,12.90,13.56,34.67,71.00,0,0,0.000000,1.000000,0.000000,1.000000,0
2,2018-01-03,12.90,12.80,24.25,80.50,0,0,0.000000,1.000000,0.781831,0.623490,0
3,2018-01-04,12.90,9.09,25.25,69.00,0,0,0.000000,1.000000,0.974928,-0.222521,0
4,2018-01-05,12.79,15.81,46.75,79.50,0,0,0.000000,1.000000,0.433884,-0.900969,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1794,2022-11-30,17.67,31.65,46.58,42.15,0,0,-0.866025,0.500000,0.781831,0.623490,0
1795,2022-12-01,22.85,32.04,38.62,25.10,0,1,-0.500000,0.866025,0.974928,-0.222521,0
1796,2022-12-02,28.70,35.24,39.72,24.95,0,2,-0.500000,0.866025,0.433884,-0.900969,0
1797,2022-12-03,19.83,23.61,32.00,13.55,0,0,-0.500000,0.866025,-0.433884,-0.900969,0


In [6]:
df_raw

,Date_time,PM25,PM10,NO2,O3,Pollution_peak,ATMO,sin_Month,cos_Month,sin_day,cos_day,confinement
0,2018-01-01,12.90,8.74,14.33,78.50,0,0,0.000000,1.000000,-0.781831,0.623490,0
1,2018-01-02,12.90,13.56,34.67,71.00,0,0,0.000000,1.000000,0.000000,1.000000,0
2,2018-01-03,12.90,12.80,24.25,80.50,0,0,0.000000,1.000000,0.781831,0.623490,0
3,2018-01-04,12.90,9.09,25.25,69.00,0,0,0.000000,1.000000,0.974928,-0.222521,0
4,2018-01-05,12.79,15.81,46.75,79.50,0,0,0.000000,1.000000,0.433884,-0.900969,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1794,2022-11-30,17.67,31.65,46.58,42.15,0,0,-0.866025,0.500000,0.781831,0.623490,0
1795,2022-12-01,22.85,32.04,38.62,25.10,0,1,-0.500000,0.866025,0.974928,-0.222521,0
1796,2022-12-02,28.70,35.24,39.72,24.95,0,2,-0.500000,0.866025,0.433884,-0.900969,0
1797,2022-12-03,19.83,23.61,32.00,13.55,0,0,-0.500000,0.866025,-0.433884,-0.900969,0


## Make Forecast


### Train the model

[`Check the documentation of greykite`](https://github.com/linkedin/greykite)

We want to use the PROPHET model from Facebook to train your model and make a prediction for future turnover

Prophet is a procedure for forecasting time series data based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects. It works best with time series that have strong seasonal effects and several seasons of historical data.

Set the model_template as 'PROPHET' to use the model from facebook.  
 Forecast your model for a horizon of 1 month (in order to predict your last month of december)



Define the `ForecastConfig` with :
- `metadata_param` : tell the model what is your time and value columns in your dataset

  -> define `time_col`, `value_col`

  -> define `train_end_date` as datetime.datetime to tell the model when to stop the training.
- `model_template` : select 'PROPHET' since is the model we want to use
- `forecast_horizon` : how many steps ahead we want to predict
- `coverage` : 95% of actuals should fall within the prediction intervals (`coverage=0.95`)

We want to use the same trainning values than our first ARIMA model (values until 1st Dec 2021) and test our prediction with the same values (month of December 2021). 

This model provides nice vizualisations if we give it all the `df` dataset and fix the training values with `train_end_date` variable. 

Also, this model will use cross-validation methods to fit and test inside the training dataset to find the best parameters.

In [7]:
# $CHA_BEGIN
config = ForecastConfig(
     metadata_param= MetadataParam(
      time_col = "Date_time",
      value_col = "ATMO",
      train_end_date = datetime.datetime(2022, 11, 1)),
     model_template="PROPHET",
     forecast_horizon=7,
     coverage=0.95 )
# $CHA_END

In [8]:
import logging
forecaster = Forecaster()
logging.getLogger('prophet').setLevel(logging.WARNING) #not printing log
logging.getLogger('cmdstanpy').setLevel(logging.WARNING) #not printing log
result = forecaster.run_forecast_config(df=df_raw, config=config) 
model_prophet = result.model

Fitting 3 folds for each of 1 candidates, totalling 3 fits


20:21:34 - cmdstanpy - INFO - Chain [1] start processing
20:21:44 - cmdstanpy - INFO - Chain [1] done processing
/Users/llm/.pyenv/versions/3.10.6/envs/PDPA-virt_env/lib/python3.10/site-packages/greykite/common/evaluation.py:418: UserWarning: y_true contains 0. MAPE is undefined.
  warnings.warn("y_true contains 0. MAPE is undefined.")
/Users/llm/.pyenv/versions/3.10.6/envs/PDPA-virt_env/lib/python3.10/site-packages/greykite/sklearn/sklearn_scorer.py:174: UserWarning: Score is undefined for this split, setting to `np.nan`.
  warnings.warn("Score is undefined for this split, setting to `np.nan`.")
/Users/llm/.pyenv/versions/3.10.6/envs/PDPA-virt_env/lib/python3.10/site-packages/greykite/common/evaluation.py:435: UserWarning: y_true contains 0. MedAPE is undefined.
  warnings.warn("y_true contains 0. MedAPE is undefined.")
/Users/llm/.pyenv/versions/3.10.6/envs/PDPA-virt_env/lib/python3.10/site-packages/greykite/sklearn/sklearn_scorer.py:174: UserWarning: Score is undefined for this spli

20:22:02 - cmdstanpy - INFO - Chain [1] start processing
20:22:03 - cmdstanpy - INFO - Chain [1] done processing
/Users/llm/.pyenv/versions/3.10.6/envs/PDPA-virt_env/lib/python3.10/site-packages/greykite/common/evaluation.py:418: UserWarning: y_true contains 0. MAPE is undefined.
  warnings.warn("y_true contains 0. MAPE is undefined.")
/Users/llm/.pyenv/versions/3.10.6/envs/PDPA-virt_env/lib/python3.10/site-packages/greykite/common/evaluation.py:435: UserWarning: y_true contains 0. MedAPE is undefined.
  warnings.warn("y_true contains 0. MedAPE is undefined.")
/Users/llm/.pyenv/versions/3.10.6/envs/PDPA-virt_env/lib/python3.10/site-packages/greykite/common/evaluation.py:483: UserWarning: y_true is constant. Correlation is not defined.
  warnings.warn("y_true is constant. Correlation is not defined.")
/Users/llm/.pyenv/versions/3.10.6/envs/PDPA-virt_env/lib/python3.10/site-packages/greykite/common/evaluation.py:418: UserWarning: y_true contains 0. MAPE is undefined.
  warnings.warn("y_t

Now that your model predicted a forecast, you will be able to access these results

### Evaluate your forecast

We can access the result of the prediction with `result.forecast` and we can evaluate the performance with `result.forecast.test_evalution`. 
Find the same metric as the Arima model and assign it to a `score_prophet` variable.

In [9]:
df_result = result.forecast.df
df_result

,Date_time,actual,forecast,forecast_lower,forecast_upper
0,2018-01-01,0,1.027395,-0.258781,2.289621
1,2018-01-02,0,0.071773,-1.248937,1.344352
2,2018-01-03,0,0.127625,-1.140777,1.422852
3,2018-01-04,0,0.491662,-0.708386,1.874029
4,2018-01-05,0,0.880529,-0.412341,2.070385
...,...,...,...,...,...
1768,2022-11-04,0,0.512566,-0.720935,1.835576
1769,2022-11-05,0,0.081827,-1.122121,1.390848
1770,2022-11-06,0,0.034743,-1.316157,1.246169
1771,2022-11-07,0,0.071702,-1.281551,1.310532


In [10]:
df_result['forecastclass']=round(df_result['forecast']).astype(int)

In [11]:
df_result.dropna(inplace=True)

In [12]:
from sklearn.metrics import accuracy_score
accuracy_score(df_result['actual'], df_result['forecastclass'])

0.5454032712915962

In [13]:
print(f"The MAPE of Prophet model is {round(result.forecast.test_evaluation['MAPE'])}%")

TypeError: type NoneType doesn't define __round__ method

Is this model better than ARIMA ?

We also have access to `result.backtest` information from the cross-validation method but we're not going to use this today. 

### Plot the forecast

Greykite library has its own fancy plot function that integrate the upper and lower bound of the prediction.

Remember that you can zoom on plotly graph

In [ ]:
result.forecast.plot()

### Custom the model with specific holidays and seasonality

Facebook Prophet model proposes us to integrate holidays and seasonality to the algorithm.

We would like to custom them based on our understanding of the data.

Try to query the table `sales` in BigQuery to find out where the higher number of sales is made. 
We will specify the model which country holidays should impact most the turnover.

Do not erase your `df` dataset, choose an other name !

<details>
    <summary><i>Hint</i></summary>
use `COUNT(*)`, `GROUP BY` and `ORDER BY` to query the top 3 countries which have the higher number of sales. 
</details>        

In [ ]:
# $CHA_BEGIN
config_holidays = ForecastConfig(
    metadata_param = MetadataParam(
      time_col = "Date_time",
      value_col = "ATMO",
      train_end_date = datetime.datetime(2022, 11, 1)),
    model_template = "PROPHET",
    forecast_horizon = 7,
    coverage = 0.95,
    model_components_param = ModelComponentsParam(
     seasonality={"yearly_seasonality": True, 'daily_seasonality':True},
     events = {
         "holiday_lookup_countries": ["France"],
         "holiday_pre_num_days": [0],
         "holiday_post_num_days": [0],
     }))


# $CHA_END

In [ ]:
result_custom = Forecaster().run_forecast_config(
    df = df_preproc,
    config = config_holidays)
model_prophet_custom = result_custom.model

Assign the MAPE to the variable `score_prophet_custom`

In [ ]:
df_result_custom = result_custom.forecast.df
df_result_custom

In [ ]:
df_result_custom['forecastclass']=round(df_result_custom['forecast']).astype(int)

In [ ]:
df_result_custom.dropna(inplace=True)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(df_result_custom['actual'], df_result_custom['forecastclass'])

In [ ]:
print(f"The MAPE of Prophet model is {round(result_custom.forecast.test_evaluation['MAPE'])}%")

In [ ]:
result_custom.forecast.plot_components()

The time serie will be the addition of all these series.